**Important**

in this notebook, i noticed that that the dataset was quite small it has only 299 observations which is quite small to perform a test train split and train a good model. Also there is a very high class imbalance in the dependent variable(DEATH_EVENT) so i performed an upsample of the dataset by SMOTING to equate the amount of classes in the dependent variable.

The following processes were carried out as Pre-processing before training models:

1. Upsampling the dataset by Smoting to increase the number of observations to 400.

2. Use of columns transformer to transform some columns using MinMaxScaler transformer.
3. perform Grid search hyper-parameter tunning for DecisionTreeClassifier and RandomForestClassifier.
4. fitting the models, making predictons and evaluating the metrics.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Series, DataFrame

In [2]:
dataset = pd.read_csv('https://raw.githubusercontent.com/gogzicole/stage-f-07-heart-failure/master/data/heart_failure_clinical_records_dataset.csv')
dataset.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [4]:
dataset.shape

(299, 13)

In [3]:
# selecting feature matrix and target variable
X = dataset.drop(columns = 'DEATH_EVENT')
y = dataset['DEATH_EVENT']
y.value_counts()

0    203
1     96
Name: DEATH_EVENT, dtype: int64

As we can see in the DEATH_EVENT column the instances of 1 and 0 are very much far apart, so creating a model using this data in this form would most likely get a high accuracy score but would fail in its precision and recall score. This model being for medical use we would have less concerne with the accuracy score but the precision and mostly recall.

This is because we want less instances of FN which is evedent of a high recall score. we also want less instances of FP which is evident of high precision, but we cant have both a high precision and high recall at the same time as they are inversely proportional, so we would also aim to have a high f1_score which is the harmonic mean of recall and precision.

i would demonstrate this by first building a model with the dataset as it is and then calculate the metrics. then i would perform Smotin and then build another model.


**First models Without Smoting**

**RandomForestclassifier**

i would be using sklearn pipelines to encapsulate the whole steps in a pipe to make the work a little bit messy and wit less blocks of code.

In [41]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import  ColumnTransformer
from sklearn.ensemble import RandomForestClassifier

In [42]:
make_column_transformer = [2,4,6,7,8,11]

pipe = Pipeline([('col_transform', ColumnTransformer(remainder = 'passthrough',
                transformers = [('scaler',MinMaxScaler(),make_column_transformer)])),
                ('grid_search',GridSearchCV(RandomForestClassifier(),
                {'max_depth':range(1,15), 'n_estimators': [50, 100,300,500],
                'max_features': ['auto', 'sqrt', 'log'], 
                'min_samples_split': [2,3,5,7,9],'min_samples_leaf': [1,2,4,6,8]},
                cv = 3, n_jobs = 2, scoring = 'accuracy', verbose = 1))])

In [43]:
Xt1 = pipe.named_steps['col_transform'].fit_transform(X)

In [44]:
x_train, x_test,y_train, y_test = train_test_split(Xt1, y, test_size = 0.2, random_state = 0)

In [45]:
pipe.fit(x_train,y_train)

Fitting 3 folds for each of 4200 candidates, totalling 12600 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   13.5s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:   54.3s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:  2.1min
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:  3.1min
[Parallel(n_jobs=2)]: Done 1246 tasks      | elapsed:  4.9min
[Parallel(n_jobs=2)]: Done 1796 tasks      | elapsed:  6.5min
[Parallel(n_jobs=2)]: Done 2446 tasks      | elapsed:  9.4min
[Parallel(n_jobs=2)]: Done 3196 tasks      | elapsed: 12.1min
[Parallel(n_jobs=2)]: Done 4046 tasks      | elapsed: 15.2min
[Parallel(n_jobs=2)]: Done 4996 tasks      | elapsed: 18.7min
[Parallel(n_jobs=2)]: Done 6046 tasks      | elapsed: 22.6min
[Parallel(n_jobs=2)]: Done 7196 tasks      | elapsed: 26.2min
[Parallel(n_jobs=2)]: Done 8446 tasks      | elapsed: 31.2min
[Parallel(n_jobs=2)]: Done 9796 tasks      | elapsed: 36.0min
[Parallel(n_jobs=2)]: Done 11246 tasks      | elapsed: 41.5mi

Pipeline(memory=None,
         steps=[('col_transform',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('scaler',
                                                  MinMaxScaler(copy=True,
                                                               feature_range=(0,
                                                                              1)),
                                                  [2, 4, 6, 7, 8, 11])],
                                   verbose=False)),
                ('grid_search',
                 GridSearchCV(cv=3, error_score=nan,
                              estimator=RandomForestClassifier(bootstrap=True,
                                                               ccp_alp...
                                                               oob_score=False,
            

In [46]:
model1 = pipe.named_steps['grid_search'].best_estimator_
model1.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=6, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=3,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [47]:
y_pred1 = model1.predict(x_test)

In [48]:
from sklearn.metrics import classification_report, recall_score, accuracy_score,precision_score, f1_score
report1 = classification_report(y_test,y_pred1)
print('The classification report for the Dataset as it is:')
print(report1)

The classification report for the Dataset as it is:
              precision    recall  f1-score   support

           0       0.81      0.95      0.88        37
           1       0.88      0.65      0.75        23

    accuracy                           0.83        60
   macro avg       0.85      0.80      0.81        60
weighted avg       0.84      0.83      0.83        60



In [49]:
recall = recall_score(y_test,y_pred1)
accuracy = accuracy_score(y_test,y_pred1)
precision = precision_score(y_test,y_pred1)
f1score = f1_score(y_test,y_pred1)
print('the Recall is:{}'.format(round(recall,4)))
print(f'the Accuracy is:{round(accuracy,4)}')
print('the Precision is: %s' %(round(precision,4)))
print(f'the F1_score is: {f1score}')

the Recall is:0.6522
the Accuracy is:0.8333
the Precision is: 0.8824
the F1_score is: 0.75


As we can see the model performed quite poorly with a precision of 88% and a recall of about 65% this is quite poor and unacceptable.

Now we would train another model using the same hyparameters on grid search and on an umsampled dataset

**First Model with SMOTING**

In [10]:
#perform Smote on X and y and reassign the new variables to x_balanced and ya_balanced
import imblearn
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=1)
X_balanced, y_balanced = smote.fit_sample(X, y)
X_balanced = DataFrame(X_balanced, columns = X.columns)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [6]:
# we can see that the numbr of observations increased from 299 to 406
# and also both instances of 1 and 0 in y are now equal
print('the shape of X is: {}'.format(X.shape))
print('the shape of y is: {}'.format(y.shape))
print('the shape of X_balanced is: {}'.format(X_balanced.shape))
print('the shape of y_balanced is: {}'.format(Series(y_balanced).shape))
print('the instances of 1 and 0 in DEATH_EVENT are:')
print(Series(y_balanced).value_counts())

the shape of X is: (299, 12)
the shape of y is: (299,)
the shape of X_balanced is: (406, 12)
the shape of y_balanced is: (406,)
the instances of 1 and 0 in DEATH_EVENT are:
1    203
0    203
dtype: int64


**Random Forest Classifier on SMOTED data**

In [12]:
# I would perform MinMaxScaling using ColumnTransform to passthrough some columns
# Also i would perform GridSearchCV for the RandomForestClassifier
# The whole process would be encapsulated in an sklearn pipeline.

make_column_transformer = [2,4,6,7,8,11] # This variable specifies the columns that would be scaled, while the rest are passthrough

pipe5 = Pipeline([('col_transform', ColumnTransformer(remainder = 'passthrough',
                transformers = [('scaler',MinMaxScaler(),make_column_transformer)])),
                ('grid_search',GridSearchCV(RandomForestClassifier(),
                {'max_depth':range(1,15), 'n_estimators': [50, 100,300,500],
                'max_features': ['auto', 'sqrt', 'log'], 
                'min_samples_split': [2,3,5,7,9],'min_samples_leaf': [1,2,4,6,8]},
                cv = 3, n_jobs = 2, scoring = 'accuracy', verbose = 1))])

Xt5 = pipe.named_steps['col_transform'].fit_transform(X_balanced) #fitting and transforming the smoted feature matrix to the transformer

x_train, x_test,y_train, y_test = train_test_split(Xt5, y_balanced, test_size = 0.2, random_state = 0)

pipe5.fit(x_train,y_train) # perfoming the GridSearchCV by calling the pipe fit method

Fitting 3 folds for each of 4200 candidates, totalling 12600 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   13.4s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:   54.5s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:  2.1min
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:  3.1min
[Parallel(n_jobs=2)]: Done 1246 tasks      | elapsed:  4.8min
[Parallel(n_jobs=2)]: Done 1796 tasks      | elapsed:  6.4min
[Parallel(n_jobs=2)]: Done 2446 tasks      | elapsed:  9.3min
[Parallel(n_jobs=2)]: Done 3196 tasks      | elapsed: 12.1min
[Parallel(n_jobs=2)]: Done 4046 tasks      | elapsed: 15.2min
[Parallel(n_jobs=2)]: Done 4996 tasks      | elapsed: 18.8min
[Parallel(n_jobs=2)]: Done 6046 tasks      | elapsed: 22.7min
[Parallel(n_jobs=2)]: Done 7196 tasks      | elapsed: 26.4min
[Parallel(n_jobs=2)]: Done 8446 tasks      | elapsed: 31.5min
[Parallel(n_jobs=2)]: Done 9796 tasks      | elapsed: 36.3min
[Parallel(n_jobs=2)]: Done 11246 tasks      | elapsed: 42.0mi

Pipeline(memory=None,
         steps=[('col_transform',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('scaler',
                                                  MinMaxScaler(copy=True,
                                                               feature_range=(0,
                                                                              1)),
                                                  [2, 4, 6, 7, 8, 11])],
                                   verbose=False)),
                ('grid_search',
                 GridSearchCV(cv=3, error_score=nan,
                              estimator=RandomForestClassifier(bootstrap=True,
                                                               ccp_alp...
                                                               oob_score=False,
            

In [17]:
model5 = pipe5.named_steps['grid_search'].best_estimator_ #Training the RandomForestClassifier with the best parameters from the search
model5.fit(x_train, y_train)
y_pred5 = model5.predict(x_test)

In [18]:
from sklearn.metrics import classification_report, recall_score, accuracy_score,precision_score, f1_score
recall5 = recall_score(y_test,y_pred5)
accuracy5 = accuracy_score(y_test,y_pred5)
precision5 = precision_score(y_test,y_pred5)
f1score5 = f1_score(y_test,y_pred5)
print('the Recall for smoted RandomForest is:{}'.format(round(recall5,4)))
print(f'the Accuracy for smoted RandomForest is:{round(accuracy5,4)}')
print('the Precision for smoted RandomForest is: %s' %(round(precision5,4)))
print(f'the F1_score for smoted RandomForest is: {round(f1score5,4)}')

the Recall for smoted RandomForest is:0.9574
the Accuracy for smoted RandomForest is:0.9024
the Precision for smoted RandomForest is: 0.8824
the F1_score for smoted RandomForest is: 0.9184


In [50]:
# saving the model to Disk
import dill
with open('RandomForest.dill','wb') as f:
  dill.dump(model5,f)

In [54]:
#checking if model is saved successfully on disk
!ls -alh RandomForest.dill

-rw-r--r-- 1 root root 298K Oct 24 00:43 RandomForest.dill


**Decision Tree Classifier on SMOTE dataset**

I would perform the same process here also, but without a pipeline so u can better understand the steps carried out using pipeline.

there would be no need to perform scaling on the dataset again as i would be using the already scaled dataset from the previous model.

In [37]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
grid6 = GridSearchCV(DecisionTreeClassifier(),{'max_depth':range(1,300),
                    'max_features': ['auto', 'sqrt', 'log'], 'min_samples_split': range(10,400,10),
                    'min_samples_leaf':[1,2,4,6,8],'max_depth':[2,4,6,8,50]},
                    cv = 3, n_jobs = 2, scoring = 'accuracy', verbose = 1)
grid6.fit(x_train,y_train)

Fitting 3 folds for each of 2925 candidates, totalling 8775 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done 2812 tasks      | elapsed:    2.8s
[Parallel(n_jobs=2)]: Done 8772 out of 8775 | elapsed:    8.1s remaining:    0.0s
[Parallel(n_jobs=2)]: Done 8775 out of 8775 | elapsed:    8.1s finished


GridSearchCV(cv=3, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=2,
             param_grid={'max_depth': [2, 4, 6, 8, 50],
                         

In [38]:
model6 = grid1.best_estimator_
model6.fit(x_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=4, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=6, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [39]:
y_pred6 = model6.predict(x_test)
from sklearn.metrics import classification_report
report6 = classification_report(y_test,y_pred6)
print(report6)

              precision    recall  f1-score   support

           0       0.83      0.86      0.85        35
           1       0.89      0.87      0.88        47

    accuracy                           0.87        82
   macro avg       0.86      0.86      0.86        82
weighted avg       0.87      0.87      0.87        82



In [40]:
recall6 = recall_score(y_test,y_pred6)
accuracy6 = accuracy_score(y_test,y_pred6)
precision6 = precision_score(y_test,y_pred6)
f1score6 = f1_score(y_test,y_pred6)
print('the Recall for smoted DecisionTree is:{}'.format(round(recall6,4)))
print(f'the Accuracy for smoted DecisionTree is:{round(accuracy6,4)}')
print('the Precision for smoted DecisionTree is: %s' %(round(precision6,4)))
print(f'the F1_score for smoted DecisionTree is: {round(f1score6,4)}')

the Recall for smoted DecisionTree is:0.8723
the Accuracy for smoted DecisionTree is:0.8659
the Precision for smoted DecisionTree is: 0.8913
the F1_score for smoted DecisionTree is: 0.8817


**Conclusion**

as we can see the RandomForest model on the SMOTE dataset has the best performance metrics and should be selected as the model of choice.

its hyperparameters are listed below

In [55]:
pipe5.named_steps['grid_search'].best_params_ 

{'max_depth': 9,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 3,
 'n_estimators': 50}